In [1]:
#Live Database
import json
from langchain_community.graphs import Neo4jGraph
def graph_db():
    graph = Neo4jGraph(
        url="neo4j+s://1eab09b0.databases.neo4j.io", username="neo4j", password="ykfXWCmzh4aIxmlj50k52ldHX7qvFvVbHN4Qmm3Z8ZA")
    return graph
live_graph = graph_db()

schema = graph_db().schema
# print(schema)

In [13]:
#TEST DATABASE
from langchain_community.graphs import Neo4jGraph
def graph_db():
    graph = Neo4jGraph(
        url="neo4j+s://d652d339.databases.neo4j.io", username="neo4j", password="S_rv6XHVT3WdLHrXfgKxEPEu7an6vsc71HFS6ZzpJfc")
    return graph

schema = graph_db().schema
test_graph = graph_db()
# print(schema)

In [14]:
import phonenumbers
from email_validator import validate_email, EmailNotValidError
from phonenumbers import NumberParseException, is_valid_number
import re

def validate_email_address(email):
    """
    Validate the email address format and provide feedback to the lead agent.
    """
    try:
        valid = validate_email(email)
        email = valid.email
        return True
    except EmailNotValidError as e:
        return f"The email address '{email}' is invalid. Please verify and provide a correct email address format."

def validate_phone_number(number: str):
    """
    Validate the phone number format including the country code and provide feedback to the lead agent.
    """
    if not number.startswith('+'):
        return "The phone number must include the country code (e.g., +1 for the US). Please provide the correct format."

    if not number[1:].replace("-", "").replace(" ", "").isdigit():
        return "The phone number contains invalid characters. Ensure that it only contains digits, spaces, or hyphens after the country code."

    try:
        phone_number = phonenumbers.parse(number)
        if is_valid_number(phone_number):
            return True
        else:
            return f"The phone number '{number}' is not valid. Please double-check the number and try again."
    except NumberParseException:
        return f"An error occurred while validating the phone number '{number}'. Please verify that it is in the correct international format."

def validate_civil_id(civil_id):
    """
    Validate the Civil ID format and provide feedback to the lead agent.
    """
    civil_id_regex = r'^\d{12}$'

    if re.fullmatch(civil_id_regex, civil_id):
        return True
    else:
        return f"The Civil ID '{civil_id}' is invalid. A valid Civil ID should contain exactly 12 digits with no other characters."


In [18]:
def verify_lead(name: str = None, email: str = None, phone: str = None, civil_id: str = None):
    """
    Verify the existence of the Customer in our Database before lead creation.
    
    Parameters:
    - name: Customer's name (optional).
    - email: Customer's email (optional).
    - phone: Customer's phone (optional).
    - civil_id: Customer's civil ID (optional).
    
    Returns:
    - Clear and actionable messages for the lead agent to understand the verification results.
    """
    def validation():
        phone_validation = True
        civil_validation = True
        email_validation = True

        if phone:
            phone_validation = validate_phone_number(phone)
            if phone_validation is not True:
                return phone_validation
        
        if civil_id:
            civil_validation = validate_civil_id(civil_id)
            if civil_validation is not True:
                return civil_validation
        
        if email:
            email_validation = validate_email_address(email)
            if email_validation is not True:
                return email_validation

        # If all validations pass, return True
        return all([phone_validation, civil_validation, email_validation])

    validated = validation()

    if validated == True:

        name_query = """
                    MATCH (c:Lead)
                    WITH c, apoc.text.levenshteinSimilarity(toLower(c.name), toLower($name)) AS similarity_score
                    WHERE similarity_score > 0.45
                    RETURN c.name AS customer_name, c.phone_number AS phone_number, c.email AS email, c.civil_id AS civil_id
                    ORDER BY similarity_score DESC
                    """

        name_contains_query = """
                    MATCH (c:Lead)
                    WHERE toLower(c.name) CONTAINS toLower($name)
                    RETURN c.name AS customer_name, 
                        c.phone_number AS phone_number, 
                        c.email AS email, 
                        c.civil_id AS civil_id
                    """
        both_query = """MATCH (c:Lead)
                        WHERE toLower(c.name) CONTAINS toLower($name)
                        WITH c
                        WITH c, apoc.text.levenshteinSimilarity(toLower(c.name), toLower($name)) AS similarity_score
                        WHERE similarity_score > 0.6
                        RETURN c.name AS customer_name, 
                            c.phone_number AS phone_number, 
                            c.email AS email, 
                            c.civil_id AS civil_id
                        ORDER BY similarity_score DESC
                        """    

        if name and all(x is None for x in (phone, civil_id, email)):

            name_result = live_graph.query(name_query, {'name': name})
            name_contains_result = live_graph.query(name_contains_query, {'name': name})
            both_result = live_graph.query(both_query, {'name': name})

            if name_result:
                return f"The provided Name is associated with the following customer(s): {[entry['customer_name'] for entry in name_result]}. Would you like to proceed with one of these customers, or would you prefer to create a new lead?"        
            else:
                return f"No matching results were found for the name '{name}'. Please review or confirm the provided details. Would you like to create a new lead instead?"

        else:
            query = """
            MATCH (l:Lead)
            WHERE toLower(l.name) = toLower($name)
            """
            conditions = []
            if phone:
                conditions.append("l.phone_number = $phone")
            if civil_id:
                conditions.append("l.civil_id = $civil_id")
            if email:
                conditions.append("l.email = $email")
            
            if conditions:
                query += " AND " + " AND ".join(conditions)

            query += """
            RETURN l.name AS lead_name, l.phone_number AS phone_number, l.civil_id AS civil_id, l.email AS email, l.id AS lead_id
            """        
            verified_result = live_graph.query(query, {
                'name': name,
                'phone': phone,
                'civil_id': civil_id,
                'email': email
            })

            if verified_result:
                return f"A customer named '{name}' already exists in our system with matching details (Phone: {verified_result[0]['phone_number']}, Email: {verified_result[0]['email']}, Civil ID: {verified_result[0]['civil_id']}). Would you like to proceed with this customer, or create a new lead?"
            
            else:        
                cypher_queries = []
                semi_verified_result = []

                if phone:
                    phone_query = """
                    MATCH (c:Lead)
                    WHERE c.phone_number = $phone 
                    RETURN c.name AS customer_name, c.phone_number AS phone_number, c.email AS email, c.civil_id AS civil_id
                    """
                    cypher_queries.append(("phone number", phone_query))

                if civil_id:
                    civil_id_query = """
                    MATCH (c:Lead) 
                    WHERE c.civil_id = $civil_id 
                    RETURN c.name AS customer_name, c.phone_number AS phone_number, c.email AS email, c.civil_id AS civil_id
                    """
                    cypher_queries.append(("civil ID", civil_id_query))

                if email:
                    email_query = """
                    MATCH (c:Lead) 
                    WHERE c.email = $email 
                    RETURN c.name AS customer_name, c.phone_number AS phone_number, c.email AS email, c.civil_id AS civil_id
                    """
                    cypher_queries.append(("email", email_query))

                for query_type, query in cypher_queries:
                    db_result = live_graph.query(query, {
                        'name': name,
                        'phone': phone,
                        'civil_id': civil_id,
                        'email': email
                    })

                    if db_result:
                        return f"I have identified that the {query_type} you provided is associated with {db_result[0]['customer_name']}. Could you please confirm or provide the correct {query_type} for {name}?"

                    else:
                        return f"I was unable to locate the {query_type} ({phone if query_type == 'phone number' else civil_id if query_type == 'civil ID' else email}) in our system. Would you like to proceed with creating a new lead?"
    else:return validated

In [20]:
verify_lead(name="shanmugam")

"The provided Name is associated with the following customer(s): ['Kannan shanmugam']. Would you like to proceed with one of these customers, or would you prefer to create a new lead?"

In [340]:
#creation tool

# model = live_graph.query("MATCH (m:Model) RETURN m", {})
model = "TIGGO8PRO"
variant = "TIGGO8 PRO 1.5L I4 PHEV PREMIUM"

def create_lead(name: str, phone: str, civil_id: str, email: str):

    if validate_email_address(email) and validate_phone_number(phone) and validate_civil_id(civil_id):
        query = """
                MERGE (l:Customer {phone_number: $mobile})
                ON CREATE SET l.createdAt = $createdAt,
                            l.id = apoc.create.uuid()
                SET l.name = $name,
                    l.email = $email,
                    l.model = $model,
                    l.variant = $variant,
                    l.civil_id = $civil_id
           
                WITH l
                MERGE (c:Lead {id: l.id})
                ON CREATE SET c.createdAt = $createdAt
                SET c.name = $name, 
                    c.phone_number = $mobile, 
                    c.email = $email, 
                    c.model = $model,
                    c.variant = $variant,
                    c.level = "High"
                MERGE (l)-[:CUSTOMER_OF_LEAD]->(c)
                
                WITH l  // Added this line
                MATCH (m:Model{name : $model})
                MERGE (c)-[:PREFERENCE]->(m)
            """
        params = {
            "name": name.capitalize(),
            "mobile": phone,
            "email":email,
            "createdAt": datetime.datetime.now().isoformat(),
            "variant" : variant,
            "model" : model,
            "civil_id" : civil_id
        }
        test_graph.query(query, params)

        return f"Successfully created lead for {name}\n View Info of {name}({phone})"


In [9]:
from verifylead import verify_lead
verify_lead(name="kannan")

"No matching results were found for the name 'kannan'. Please review or confirm the provided details. Would you like to create a new lead instead?"

In [2]:

import time
start = time.time()
query = live_graph.query("""MATCH (c:Lead) RETURN c.name AS customer_name""")
names_list = [name for i in query for name in [i['customer_name']]]
print(time.time() - start)

0.12942838668823242


In [3]:
def split_names(names):
    return [name.split(' ', 1) if ' ' in name else name for name in names]

names_list = split_names(names_list)

In [4]:
names_list.append(["Ramesh","sekar"])

In [5]:
from pyjarowinkler import distance

def find_similar_names_jaro_winkler(user_name, names_list, threshold=0.7):
    similar_names = []
    for name in names_list:
        if isinstance(name, str):
            similarity = distance.get_jaro_distance(user_name, name)
            if similarity >= threshold:
                similar_names.append((name, similarity))
        else:
            for i in name:
                similarity = distance.get_jaro_distance(user_name, i)
                if similarity >= threshold:
                    similar_names.append((' '.join(name), similarity))
    return similar_names

# Example usage



In [6]:
find_similar_names_jaro_winkler("rsekra", names_list, threshold=0.8)

[('Vikram Sekar', 0.88), ('Ramesh sekar', 0.88)]

In [9]:
from verifylead import verify_lead
verify_lead(name="sekra",email="",phone=None,civil_id=None)